In [5]:
import numpy as np
import scipy.special
from scipy.signal import convolve2d

In [22]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([[1,1]])
c = convolve2d(a,b,mode='same')
c

array([[ 1,  3,  5],
       [ 4,  9, 11],
       [ 7, 15, 17]])

In [8]:
a**2

array([[ 1,  4],
       [ 9, 16]])

In [9]:
np.square(a)

array([[ 1,  4],
       [ 9, 16]])

In [11]:
np.array([a,a])

array([[[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]]])

In [24]:
np.array(list(map(np.square, np.array([[a,a],[a,a]]))))

array([[[[ 1,  4,  9],
         [16, 25, 36],
         [49, 64, 81]],

        [[ 1,  4,  9],
         [16, 25, 36],
         [49, 64, 81]]],


       [[[ 1,  4,  9],
         [16, 25, 36],
         [49, 64, 81]],

        [[ 1,  4,  9],
         [16, 25, 36],
         [49, 64, 81]]]])